# helper

> Helper functions 

In [93]:
#| default_exp helper.__init__

In [94]:
#| export
from __future__ import annotations
import random
from typing import Callable, Hashable, TypeVar
from itertools import zip_longest
import string

import Levenshtein
from Levenshtein import distance
from thefuzz import fuzz
import fuzzywuzzy



In [95]:
import operator


from fastcore.test import *

In [96]:
#| export
def substring_generator(input_string: str):
    length = len(input_string)
    for i in range(length):
        for j in range(i + 1, length + 1):
            yield input_string[i:j]


def sublist_generator(input_list: list):
    length = len(input_list)
    for i in range(length):
        for j in range(i + 1, length + 1):
            yield input_list[i:j]

In [97]:
#| export
def is_punctuation(
        char: str # A str of length 1
        ):
    return char in '.!?,:;'

def is_not_space_and_not_punc(
        char: str # A str of length 1
        ):
    return not is_punctuation(char) and not char.isspace()

In [98]:
assert is_punctuation('.')
assert not is_punctuation('1')
assert not is_not_space_and_not_punc('.')
assert is_not_space_and_not_punc('a')
assert not is_not_space_and_not_punc(' ')
assert not is_not_space_and_not_punc('\t')

In [99]:
#| export
def split_string_at_indices(s: str, indices: list[int]) -> list[str]:
    return [s[i:j] for i, j in zip([0] + indices, indices + [None])]

In [100]:
s = "Hello World! How are you?"
indices = [5, 12, 16]
result = split_string_at_indices(s, indices)
print(result)
test_eq(result, ['Hello', ' World!', ' How', ' are you?'])
# Output: ['Hello', ' World!', ' How', ' are you?']

['Hello', ' World!', ' How', ' are you?']


In [101]:
s = "Hello World! How are you?"
indices = [0, 5, 12, 16]
result = split_string_at_indices(s, indices)
print(result)
test_eq(result, ['', 'Hello', ' World!', ' How', ' are you?'])
# Output: ['Hello', ' World!', ' How', ' are you?']

['', 'Hello', ' World!', ' How', ' are you?']


## Split list into chunks

In [102]:
#| export
def split_list_into_chunks(original_list, split_ratio=0.75):
    total_length = len(original_list)
    target_length = int(total_length * split_ratio)
    
    chunks = []
    current_index = 0
    
    while current_index < total_length:
        chunk_size = random.randint(1, min(5, total_length - current_index))
        chunks.append(original_list[current_index:current_index + chunk_size])
        current_index += chunk_size
    
    random.shuffle(chunks)
    
    list1, list2 = [], []
    current_length = 0
    
    for chunk in chunks:
        if current_length + len(chunk) <= target_length:
            list1.extend(chunk)
            current_length += len(chunk)
        else:
            list2.extend(chunk)
    
    return list1, list2


In [103]:
original_list = list(range(100))
list_75, list_25 = split_list_into_chunks(original_list)

print(f"Original list length: {len(original_list)}")
print(f"75% list length: {len(list_75)}")
print(f"25% list length: {len(list_25)}")



Original list length: 100
75% list length: 74
25% list length: 26


## Get top counted items in a dict

In [104]:
#| export
K = TypeVar('K', bound=Hashable)  # Generic type for keys

def get_top_counted_items(
        data: dict[K, int],
        top_percentile: float = 0.10,
        max_number: int = 5,
    ) -> list[K]:
        """Returns top items from a dictionary where keys are of type K and values are counts.
        
        Args:
            data: Dictionary with keys of type K and integer counts as values
            top_percentile: Percentage of top items to return (default: 0.10)
            max_number: Maximum number of items to return (default: 5)
            
        Returns:
            List of keys with the highest counts, maintaining the original key type
        """
        sorted_items = sorted(data.items(), key=lambda x: x[1], reverse=True)
        num_items = max(1, min(int(len(data) * top_percentile), max_number))
        return [item[0] for item in sorted_items[:num_items]]


In [105]:
from fastcore.test import test, test_eq
from typing import Dict, List, TypeVar
from collections.abc import Hashable

K = TypeVar('K', bound=Hashable)

def get_top_counted_items(
    data: Dict[K, int],
    top_percentile: float = 0.10,
    max_number: int = 5,
) -> List[K]:
    sorted_items = sorted(data.items(), key=lambda x: x[1], reverse=True)
    num_items = max(1, min(int(len(data) * top_percentile), max_number))
    return [item[0] for item in sorted_items[:num_items]]

# --- Tests ---
# Test 1: Default behavior (10% with max 5)
test_eq(
    get_top_counted_items({'a':5, 'b':3, 'c':10}),
    ['c']  # 3*0.10=0.3 → returns top 1
)

# Test 2: max_number override
test_eq(
    get_top_counted_items({1:100, 2:90, 3:80, 4:70}, top_percentile=1.0, max_number=2),
    [1, 2]
)

# Test 3: top_percentile override
test_eq(
    get_top_counted_items({'x':1, 'y':2, 'z':3}, top_percentile=0.67),
    ['z', 'y']  # 3*0.67=2.01 → returns top 2
)

# Test 4: Edge case - empty dict
test_eq(get_top_counted_items({}), [])

# Test 5: Edge case - single item
test_eq(get_top_counted_items({'x':1}), ['x'])

# Test 6: Hashable non-string keys
class City:
    def __init__(self, name): self.name = name
    def __hash__(self): return hash(self.name)
    
nyc, london = City('NYC'), City('London')
test_eq(
    get_top_counted_items({nyc:8, london:12}),
    [london]
)

# Test 7: Tuple keys (hashable)
test_eq(
    get_top_counted_items({(1,2):10, (3,4):5}),
    [(1,2)]
)

## Whether one string is contained in another

In [ ]:
#| export
# def char_substitution_cost(c1: str, c2: str) -> float:
#     if c1 == c2 or c1 is None or c2 is None:
#         return 0
    
#     if c1 not in string.ascii_letters or c2 not in string.ascii_letters:
#         return 1.0
    
#     c1, c2 = c1.lower(), c2.lower()
#     letter_dist = abs(ord(c1) - ord(c2)) / 25
#     return min(letter_dist * 0.8, 0.8)

def char_substitution_cost(c1: str, c2: str) -> float:
    if c1 == c2 or c1 is None or c2 is None:
        return 0
    
    if c1 not in string.ascii_letters or c2 not in string.ascii_letters:
        return 1.0
    
    c1, c2 = c1.lower(), c2.lower()
    letter_dist = abs(ord(c1) - ord(c2))
    
    # Non-linear penalty curve
    if letter_dist <= 5:
        return (letter_dist / 5) * 0.5  # 0-0.5 penalty for 0-5 distance
    else:
        return min(0.5 + (letter_dist - 5)/20, 0.8)  # 0.5-0.8 penalty for >5 distance

def partial_ratio_with_alphabet(s1: str, s2: str) -> float:
    """Alphabet-aware partial ratio implementation."""
    if not s1 or not s2:
        return 0.0  # or 100.0 if you consider empty strings a perfect match
    # Ensure s1 is the shorter string
    if len(s1) > len(s2):
        s1, s2 = s2, s1
    
    len1, len2 = len(s1), len(s2)
    best_score = 0
    
    # Slide shorter string over longer string
    for i in range(len2 - len1 + 1):
        substring = s2[i:i+len1]
        
        # Calculate custom Levenshtein distance
        d = [[0] * (len1 + 1) for _ in range(2)]
        for j in range(len1 + 1):
            d[0][j] = j
            
        for j in range(1, len1 + 1):
            current_char = substring[j-1]
            d[1][0] = j
            
            for k in range(1, len1 + 1):
                cost = char_substitution_cost(s1[k-1], current_char)
                d[1][k] = min(
                    d[0][k] + 1,       # deletion
                    d[1][k-1] + 1,     # insertion
                    d[0][k-1] + cost   # substitution
                )
            
            # Swap rows for memory efficiency
            d[0], d[1] = d[1], d[0]
        
        raw_distance = d[0][len1]
        similarity = 1 - (raw_distance / len1)
        best_score = max(best_score, similarity)
    
    return best_score * 100


In [118]:
# Test Cases
print(partial_ratio_with_alphabet("T M", "X: N → T N"))  # ~85-92
print(partial_ratio_with_alphabet("p → q", "p → d"))     # ~75-85
print(partial_ratio_with_alphabet("ABC", "ABD"))         # ~80-90
print(partial_ratio_with_alphabet("M", "N"))             # ~92-96

96.66666666666667
84.0
96.66666666666667
90.0


In [ ]:
#| export


def latex_str_in_latex_str_fuzz_metric(
    substring_candidate: str,
    superstring_candidate: str
) -> float:
    """Fuzzy containment metric without length checks."""
    
    # Calculate components
    partial_sim_fuzz = fuzz.partial_ratio(substring_candidate, superstring_candidate) / 100
    partial_sim_math = partial_ratio_with_alphabet(substring_candidate, superstring_candidate) / 100
    
    # Add containment penalty factor
    containment_penalty = (
        1.0 if len(substring_candidate) <= len(superstring_candidate)
        else (len(superstring_candidate) / len(substring_candidate)) ** 2  # Quadratic penalty
    )
    
    # Combined score with dynamic weighting
    combined_score = 0.2 * partial_sim_fuzz + 0.8 * partial_sim_math
    
    # Apply fuzzy containment logic
    return combined_score * containment_penalty


In [142]:
short_str = r"\dim M"
long_str = r"M"
print(latex_str_in_latex_str_fuzz_metric(short_str, long_str))

0.027777777777777776


In [143]:
#| hide
test_superstring_candidates = ['M', 'X: M \\rightarrow T M', '\\left(U,\\left(x^{i}\\right)\\right)', 'M', 'X', 'U', '\\left(x^{i}, v^{i}\\right)', '\\pi^{-1}(U) \\subseteq T M', '\\left(U,\\left(x^{i}\\right)\\right)', 'X: M \\rightarrow T M', 'U', '\\hat{X}(x)=\\left(x^{1}, \\ldots, x^{n}, X^{1}(x), \\ldots, X^{n}(x)\\right)', 'X^{i}', 'i', 'X', 'x^{i}', 'X', 'U']

test_substring_candidates = [r'\dim M']

for substring_candidate in test_substring_candidates:
    for superstring_candidate in test_superstring_candidates:
        print(substring_candidate, superstring_candidate)
        print(latex_str_in_latex_str_fuzz_metric(substring_candidate, superstring_candidate))


\dim M M
0.027777777777777776
\dim M X: M \rightarrow T M
0.4726666666666667
\dim M \left(U,\left(x^{i}\right)\right)
0.44066666666666665
\dim M M
0.027777777777777776
\dim M X
0.004444444444444444
\dim M U
0.007777777777777777
\dim M \left(x^{i}, v^{i}\right)
0.4726666666666667
\dim M \pi^{-1}(U) \subseteq T M
0.4393333333333333
\dim M \left(U,\left(x^{i}\right)\right)
0.44066666666666665
\dim M X: M \rightarrow T M
0.4726666666666667
\dim M U
0.007777777777777777
\dim M \hat{X}(x)=\left(x^{1}, \ldots, x^{n}, X^{1}(x), \ldots, X^{n}(x)\right)
0.4793333333333335
\dim M X^{i}
0.0944444444444444
\dim M i
0.027777777777777776
\dim M X
0.004444444444444444
\dim M x^{i}
0.0944444444444444
\dim M X
0.004444444444444444
\dim M U
0.007777777777777777


In [144]:
short_str = r"\mathcal{O}_X"
long_str = r"H_{\mathcal{O}_X}"
print(latex_str_in_latex_str_fuzz_metric(short_str, long_str))

short_str = r"(U, (x_i))"
long_str = r"X: N \rightarrow T N"
print(latex_str_in_latex_str_fuzz_metric(short_str, long_str))

short_str = r"T M"
long_str = r"X: N \rightarrow T N"
print(latex_str_in_latex_str_fuzz_metric(short_str, long_str))

short_str = r"\mathcal{O}_X"
long_str = r"\mathscr{O}_X"
print(latex_str_in_latex_str_fuzz_metric(short_str, long_str))

short_str = r"\Omega_X"
long_str = r"\mathscr{O}_X"
print(latex_str_in_latex_str_fuzz_metric(short_str, long_str))

short_str = r"F_{1}^\times\cdots\times"
long_str = r"\left(x^{i}, v^{i}\right)"
print(latex_str_in_latex_str_fuzz_metric(short_str, long_str))

short_str = r"T^{*} M"
long_str = r"X: M \rightarrow T M"
print(latex_str_in_latex_str_fuzz_metric(short_str, long_str))

1.0
0.22
0.9073333333333334
0.8746153846153847
0.451
0.25866666666666666
0.4060000000000001


0.44000000000000006

In [112]:
#| export 
def latex_str_is_likely_in_latex_str(
        substring_candidate: str, #  
        superstring_candidate: str,
        metric: Callable[[str, str], float] = latex_str_in_latex_str_fuzz_metric,
        threshold: float = 0.8, # The metric needs to achieve at least this threshold for `substring candidate` to be considered to be in `superstring_candidate`.
        ) -> bool:
    score = metric(substring_candidate, superstring_candidate)
    return score > threshold

In [113]:
short_str = r"\mathcal{O}_X"
long_str = r"\Omega_{X}"
latex_str_is_likely_in_latex_str(short_str, long_str)

False